# PWS: Machine learning in de zorg
Santos van der Wansem en Mina Qayumzada




---


> Acuut nierfalen is een complexe aandoening die vaak voorkomt bij volwassenen op de intensive care-afdeling. Acuut nierfalen wordt gekenmerkt door een abrupte (binnen enkele uren) afname van de nierfunctie, wat zowel structurele schade aan de nieren als functieverlies inhoudt. Dit leidt tot een ophoping van afvalproducten in het bloed en verhindert de nieren om de juiste vochtbalans in het lichaam te handhaven. Acuut nierfalen is een ernstig syndroom vanwege de gerelateerde morbiditeit en mortaliteit. Er zijn echter een paar markers voor acuut nierfalen, deze markers zullen voor dit machine learning model de basis vormen voor de manier waarop het model de voorspellingen zal doen.

> Met dank aan Jerina Vries en Sonja Steiger, onze begeleiders

# Usage

This model consists of three parts:

- The first part is where the data from the MIMIC-iii dataset gets transformed into a dataset that can be interpreted by the machine learning algorithms. If you do not have access to the MIMIC-iii database, you can skip this step and simply download a subset of the MIMIC-iii database, that is ready to be used for training the machine learning algorithm. Also please note, that if you do have access to the MIMIC-iii dataset, you should change the paths in the code to the path to your own MIMIC-iii files. On initiation, all file paths will be set to a certain value that may not correspond to the correct location of your MIMIC-iii files.
- The second part is where the data is used to train a machine learning model. There are currently five algorithms that get trained with the data: K-nearest-neighbors, Naive bayes, Logistic regression, Support vector machine and a Neural network.
- The third part is where the trained model (in this case the KNN model but you may select a different model) gets downloaded, and then loaded to showcase to the user how this model can be implemented in other environments.

> To gain access to an already trained version of the KNN algorithm, or to the pre-made dataset, simply head over to the github of [@santosvdw](https://github.com/santosvdw/pws) and download the desired files.

## Dataset

This model uses the MIMIC-iii dataset. MIMIC-III is a large, freely-available database comprising deidentified health-related data associated with over forty thousand patients who stayed in critical care units of the Beth Israel Deaconess Medical Center between 2001 and 2012. The database includes information such as demographics, vital sign measurements made at the bedside (~1 data point per hour), laboratory test results, procedures, medications, caregiver notes, imaging reports, and mortality (including post-hospital discharge).

Johnson, A., Pollard, T., & Mark, R. (2016). MIMIC-III Clinical Database (version 1.4). PhysioNet. https://doi.org/10.13026/C2XW26

# Data selection/manipulation

In [ ]:
# Install packages
import pandas as pd
import math
from dateutil.relativedelta import relativedelta
from datetime import datetime, date

## Selecting relevant variables

In [ ]:
# ------------
# Selecting relevant variables
# ------------

# Select all diagnoses events
df = pd.read_csv('/content/drive/MyDrive/MIMIC/DIAGNOSES_ICD.csv')

In [ ]:
# Select all diagnoses events
df = pd.read_csv('/content/drive/MyDrive/MIMIC/DIAGNOSES_ICD.csv')

# Manually selected AKI related codes
aki_codes = ["66930","66932", "66934","5845","5846","5847","5848","5849"]

# Select all AKI related diagnoses
aki = df.loc[df["ICD9_CODE"].isin(aki_codes)]

# Select all patients with AKI related diagnosis
patients = []
for index, row in aki.iterrows():
  patients.append(row['SUBJECT_ID'])

In [ ]:
# ------------
# Select all lab events for AKI-diagnosed patients
# ------------
data = pd.read_csv('/content/drive/MyDrive/MIMIC/LABEVENTS.csv')

In [ ]:
# Select all measurements of AKI-related lab variables
markers = [50912,51006,43175]
data = data.loc[data['ITEMID'].isin(markers)]

## Transforming the dataset

In [ ]:
# ------------
# Transforming the dataset
# ------------

# Dropping incomplete and irrelevant data
data = data[data['HADM_ID'].notna()]
data = data.drop(columns=["VALUE", "CHARTTIME", "FLAG"])

# OPTIONAL BUT RECOMMENDED: SELECT A SUBSET OF THE
# DATASET TO SPEED UP THE TRANSFORMATIONPROCESS
# (OTHERWISE IT MAY TAKE HOURS)

# data = data.head(400000)

In [ ]:
# ------------
# Selecting vector variables
# ------------

In [ ]:
# Rotating table on its axis
data[markers] = -1
data = data.drop(columns=["VALUEUOM", "ROW_ID"])

In [ ]:
# ------------
# Selecting urine output
# ------------
out = pd.read_csv('/content/drive/MyDrive/MIMIC/OUTPUTEVENTS.csv')
out = out.drop(columns=['ICUSTAY_ID', "CHARTTIME", "STORETIME", 'CGID', 'STOPPED', "NEWBOTTLE", "ISERROR"])

# Select urine output from file
out = out.loc[out['ITEMID']==43175]
out = out.loc[out['VALUE'].notna()]

In [ ]:
# Add all registered urine outputs to data table
for index, row in out.iterrows():
  new = [row['SUBJECT_ID'], row['HADM_ID'], 43175,0,-1,-1, row["VALUE"]]
  data.loc[len(data.index)+1] = new

In [ ]:
# Selecting values for serum creatinine and blood urea nitrogen
for i in markers:
  for index, row in data.iterrows():
    if row["ITEMID"] == i:
      data.loc[data["HADM_ID"] == row["HADM_ID"], i] = row["VALUENUM"]

In [ ]:
# Adding output value column "AKI"
data["AKI"] = 0
data.loc[data["SUBJECT_ID"].isin(patients), "AKI"] = 1

In [ ]:
# ------------
# Calculating age
# ------------

# Read admissions file and drop irrelevant columns
adm = pd.read_csv('/content/drive/MyDrive/MIMIC/ADMISSIONS.csv')
adm = adm.drop(columns=['DEATHTIME', 'DISCHTIME', 'ADMISSION_LOCATION', 'DISCHARGE_LOCATION', 'INSURANCE', 'LANGUAGE',
                        'RELIGION', 'HOSPITAL_EXPIRE_FLAG','HAS_CHARTEVENTS_DATA', 'EDREGTIME', 'EDOUTTIME', 'MARITAL_STATUS'])
# Read patients file and drop irrelevant columns
pat = pd.read_csv('/content/drive/MyDrive/MIMIC/PATIENTS.csv')
pat = pat.drop(columns=['DOD', 'DOD_HOSP', 'DOD_SSN', "EXPIRE_FLAG"])

In [ ]:
# Register time of birth in data table
for index, row in pat.iterrows():
  data.loc[data['SUBJECT_ID'] == row['SUBJECT_ID'], 'DOB'] = row['DOB']

In [ ]:
# Register time of admission in data table
for index, row in adm.iterrows():
  data.loc[data['SUBJECT_ID'] == row['SUBJECT_ID'], 'ADMITTIME'] = row['ADMITTIME']

In [ ]:
# Register age in patients table
for index, row in data.iterrows():
  age = rdelta = relativedelta(datetime.strptime(row['ADMITTIME'], '%Y-%m-%d %H:%M:%S').date(), datetime.strptime(row['DOB'],
                                                                                                                  '%Y-%m-%d %H:%M:%S').date())
  if age.years > 100:
    age.years = 90
  data.loc[data["SUBJECT_ID"] == row["SUBJECT_ID"], "AGE"] = age.years

In [ ]:
# Drop infant patients
neo = data.loc[data['AGE'] < 5].index
data = data.drop(neo)

In [ ]:
# Dropping irrelevant data
data = data.drop_duplicates(subset="HADM_ID", keep='last')
data = data.drop(columns=["HADM_ID", "ITEMID", "VALUENUM", "SUBJECT_ID", "ADMITTIME", "DOB"])

# Rearranging columns
cols = list(data.columns.values)
data = data[cols[0:3]+[cols[4]]+[cols[3]]]

In [ ]:
# ------------
# OPTIONAL: DOWNLOAD SUBSET OF DATA FOR FUTURE USE (SAVES TIME)
# ------------

# data.to_csv('dataset.csv', encoding='utf-8', index=False)

# Creating the model

In [ ]:
# ------------
# Creating the model
# ------------

# Install packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import RandomOverSampler
from sklearn.impute import SimpleImputer
from sklearn import metrics

In [ ]:
# ------------
# OPTIONAL: ONLY ENABLE IF YOU CHOSE TO DOWNLOAD A SUBSET OF THE DATA
# ------------

# data = pd.read_csv('data.csv')

In [ ]:
# Reviewing the data
cols = list(data.columns.values)
for label in cols[:-1]:
  plt.hist(data[data["AKI"]==1][label], color="blue", label="AKI", alpha=0.7, density=True)
  plt.hist(data[data["AKI"]==0][label], color="red", label="NO AKI", alpha=0.7, density=True)
  plt.ylabel("Probability")
  plt.xlabel(label)
  plt.legend()
  plt.show()

## Splitting the dataset

In [ ]:
# ------------
# Splitting the dataset
# ------------
train, valid, test = np.split(data.sample(frac=1), [int(0.6*len(data)), int(0.8*len(data))])

In [ ]:
def scale_dataset(dataframe, oversample=False):
  x = dataframe[dataframe.columns[:-1]].values
  y = dataframe[dataframe.columns[-1]].values

  scaler = StandardScaler()
  x = scaler.fit_transform(x)

  if oversample:
    ros = RandomOverSampler()
    x, y = ros.fit_resample(x, y)

  data = np.hstack((x, np.reshape(y, (-1,1))))

  return data, x, y

In [ ]:
# Create training, validation and test sets
train, x_train, y_train = scale_dataset(data, oversample=True)
valid, x_valid, y_valid = scale_dataset(data, oversample=False)
test, x_test, y_test = scale_dataset(data, oversample=False)

In [ ]:
# Get rid of NaN values
imp = SimpleImputer(missing_values=np.nan, strategy='mean')

# Transforming NaN values
imp.fit(x_train)
x_train = imp.transform(x_train)

imp.fit(x_test)
x_test = imp.transform(x_test)

## kNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report

In [ ]:
# ------------
# Create KNN model
# ------------
knn_model = KNeighborsClassifier(n_neighbors=1)
knn_model.fit(x_train, y_train)
y_pred = knn_model.predict(x_test)

In [ ]:
# ------------
# Print model metrics
# ------------

print("n=1")
print(classification_report(y_test, y_pred))

# False positives and true positive rates
fpr, tpr, _ = metrics.roc_curve(y_test, y_pred)

# Calculate AUC
auc = metrics.roc_auc_score(y_test, y_pred)
print(auc)

# Plot ROC curve
plt.plot(fpr,tpr,label="AUC="+str(auc))
plt.title(f"K Nearest Neighbors, n=1")
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.legend(loc=4)
plt.show()

n=1
              precision    recall  f1-score   support

           0       0.99      0.98      0.99      2680
           1       0.96      0.97      0.97      1190

    accuracy                           0.98      3870
   macro avg       0.98      0.98      0.98      3870
weighted avg       0.98      0.98      0.98      3870



## Naive Bayes

In [ ]:
from sklearn.naive_bayes import  GaussianNB

In [ ]:
# ------------
# Create NB model
# ------------

nb_model = GaussianNB()
nb_model.fit(x_train, y_train)
y_pred = nb_model.predict(x_test)

In [ ]:
# ------------
# Print model metrics
# ------------

print(classification_report(y_test, y_pred))

# False positives and true positive rates
fpr, tpr, _ = metrics.roc_curve(y_test, y_pred)

# Calculate AUC
auc = metrics.roc_auc_score(y_test, y_pred)
print(auc)

# Plot ROC curve
plt.plot(fpr,tpr,label="AUC="+str(auc))
plt.title("Naive Bayes")
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.legend(loc=4)
plt.show()

## Logistic regression

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
# ------------
# Create Logistic Regression model
# ------------

lg_model = LogisticRegression()
lg_model = lg_model.fit(x_train, y_train)
y_pred = lg_model.predict(x_test)

In [ ]:
# ------------
# Print model metrics
# ------------

print(classification_report(y_test, y_pred))

# False positives and true positive rates
fpr, tpr, _ = metrics.roc_curve(y_test, y_pred)

# Calculate AUC
auc = metrics.roc_auc_score(y_test, y_pred)
print(auc)

# Plot ROC curve
plt.plot(fpr,tpr,label="AUC="+str(auc))
plt.title("Logistic Regression")
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.legend(loc=4)
plt.show()

## SVM

In [ ]:
from sklearn.svm import SVC

In [ ]:
# ------------
# Create Logistic Regression model
# ------------

svm_model = SVC()
svm_model = svm_model.fit(x_train, y_train)
y_pred = svm_model.predict(x_test)

In [ ]:
# ------------
# Print model metrics
# ------------

print(classification_report(y_test, y_pred))

# False positives and true positive rates
fpr, tpr, _ = metrics.roc_curve(y_test, y_pred)

# Calculate AUC
auc = metrics.roc_auc_score(y_test, y_pred)
print(auc)

# Plot ROC curve
plt.plot(fpr,tpr,label="AUC="+str(auc))
plt.title("Support Vector Machine")
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.legend(loc=4)
plt.show()

## Neural network

In [ ]:
import tensorflow as tf

In [ ]:
# Plot NN history
def plot_history(history):
  fig, (ax1, ax2) = plt.subplots(1,2, figsize=(10,4))
  ax1.plot(history.history['loss'], label='loss')
  ax1.plot(history.history['val_loss'], label='val_loss')
  ax1.set_xlabel('Epoch')
  ax1.set_ylabel('Binary crossentropy')
  ax1.grid(True)

  ax2.plot(history.history['accuracy'], label='accuracy')
  ax2.plot(history.history['val_accuracy'], label='val_accuracy')
  ax2.set_xlabel('Epoch')
  ax2.set_ylabel('Accuracy')
  ax2.grid(True)
  plt.show()

In [ ]:
# Train NN model
def train_model(x_train, y_train, num_nodes, dropout_prob, lr, batch_size, epochs):
  nn_model = tf.keras.Sequential([
      tf.keras.layers.Dense(num_nodes, activation='relu', input_shape=(4,)),
      tf.keras.layers.Dropout(dropout_prob),
      tf.keras.layers.Dense(num_nodes, activation='relu'),
      tf.keras.layers.Dropout(dropout_prob),
      tf.keras.layers.Dense(1, activation='sigmoid')
  ])

  nn_model.compile(optimizer=tf.keras.optimizers.Adam(lr), loss='binary_crossentropy', metrics=['accuracy'])

  history = nn_model.fit(x_train, y_train, epochs=125, batch_size=32, validation_split=0.2, verbose=0)

  return nn_model, history

In [ ]:
# Train model using best arguments for this dataset
model, history = train_model(x_train, y_train, 64, 0, 0.001, 32, 100)

In [ ]:
# ------------
# Print model metrics
# ------------

plot_history(history)

y_pred = model.predict(x_test)
fpr, tpr, _ = metrics.roc_curve(y_test, y_pred)

# Calculate AUC
auc = metrics.roc_auc_score(y_test, y_pred)
print(auc)

# Plot ROC curve
plt.plot(fpr,tpr,label="AUC="+str(auc))
plt.title("NN")
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.legend(loc=4)
plt.show()

# Evaluate loss
val_loss = model.evaluate(x_valid, y_valid)
print(val_loss)

# Implementing the model

In [ ]:
import joblib

In [ ]:
# ------------
# Downloading the model
# ------------

joblib.dump(knn_model, 'aki_diagnoser.joblib')

['aki_diagnoser.joblib']

In [ ]:
# ------------
# Using the model
# ------------

model = joblib.load('aki_diagnoser.joblib')

In [ ]:
# ------------
# Making predictions by using the model
# ------------

prediction = model.predict([[1.5,11,-1,75]])
print(prediction)